<p align="center">
  <img src="https://huggingface.co/speakleash/Bielik-7B-Instruct-v0.1/raw/main/speakleash_cyfronet.png">
</p>

# Bielik Structured output vLLM + Outlines
### Po co structured output:

Pozwala na uzyskanie ustrukturyzowanych danych z modelu LLM, co ułatwia ich przetwarzanie i integrację z innymi systemami.

Jest kilka rozwiązań które pozwalają to uzyskać jak np:
- [llama.cpp](https://github.com/ggerganov/llama.cpp) i [grammar](https://github.com/ggerganov/llama.cpp/blob/master/grammars/README.md)
- [Instructor](https://python.useinstructor.com/).

Dojrzałe produkcyjnie rozwiązanie to [vLLM](https://github.com/vllm-project/vllm) + [Outlines](https://dottxt-ai.github.io/outlines/).

### Jak działa Outlines w skrócie
Outlines wykorzystuje [automaty](https://dottxt-ai.github.io/outlines/reference/generation/structured_generation_explanation/) do generowania tekstu w oparciu o zdefiniowane wzorce.

Proces polega na tym, że model językowy generuje tekst token po tokenie, ale tylko legalne tokeny (zgodne ze wzorcem) są brane pod uwagę na każdym kroku.

Przykładowo, jeśli wzorzec ma opisywać liczby całkowite i dziesiętne, automaty przeprowadzają analizę, jakie tokeny (cyfry, kropki, itp.) są dozwolone w danym momencie.

Outlines modyfikuje prawdopodobieństwa tokenów, eliminując te niezgodne z wzorcem, co zapewnia precyzyjne i kontrolowane generowanie tekstu przez model.

### Wymagania do uruchomienia tego notebooka:
GPU min 24 GB vRam - np L4 na Google Colab Pro albo [lightning.ai](https://lightning.ai/)


In [ ]:
%%capture
!pip install vllm outlines

In [ ]:
from enum import Enum
from typing import List, Optional, Union

from IPython.display import display, Markdown
from huggingface_hub import notebook_login
from outlines import generate, models
from pydantic import BaseModel, Field, constr
from transformers import AutoTokenizer

### Do pobrania modelu z Hugging Face potrzebny jest [token](https://huggingface.co/docs/hub/en/security-tokens)


In [ ]:
notebook_login()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("speakleash/Bielik-11B-v2.2-Instruct")

### Na potrzeby demo użyjemy kwantyfikowany model AWQ

In [ ]:
model = models.vllm("speakleash/Bielik-11B-v2.2-Instruct-AWQ", quantization="awq")
# model = models.vllm("speakleash/Bielik-11B-v2.2-Instruct")  # use A100 GPU at least to load full model

INFO 09-26 09:07:42 awq_marlin.py:93] Detected that the model can run with awq_marlin, however you specified quantization=awq explicitly, so forcing awq. Use quantization=awq_marlin for faster inference
WARNING 09-26 09:07:42 config.py:330] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 09-26 09:07:42 llm_engine.py:213] Initializing an LLM engine (v0.6.0) with config: model='speakleash/Bielik-11B-v2.2-Instruct-AWQ', speculative_config=None, tokenizer='speakleash/Bielik-11B-v2.2-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_co

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 09-26 09:09:20 model_runner.py:926] Loading model weights took 5.7895 GB
INFO 09-26 09:09:34 gpu_executor.py:122] # GPU blocks: 3292, # CPU blocks: 1310
INFO 09-26 09:09:41 model_runner.py:1217] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-26 09:09:41 model_runner.py:1221] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-26 09:10:17 model_runner.py:1335] Graph capturing finished in 36 secs.


### Tekst na którym będziemy testować ekstrakcję treści do formatu JSON

In [ ]:
text = """
Każdy z hobbitów miał plecak, a w nim najpotrzebniejsze rzeczy.
Frodo niósł trzy zapasowe koszule, dwa płaszcze i parę koców.
Sam miał cztery bochenki chleba, sześć kawałków suszonego mięsa i kilka ciastek,
a Merry wziął bukłak z wodą, dwie latarnie i dziesięć świec.
"""

### Pydantic i definicja struktury którą użyje model do odpowiedzi

In [ ]:
class CharactersNames(Enum):
    Frodo = "Frodo"
    Sam = "Sam"
    Merry = "Merry"


class Item(BaseModel):
    name: str
    quantity: Optional[Union[int, str]]


class Character(BaseModel):
    name: CharactersNames
    items: List[Item]


class Company(BaseModel):
    characters: List[Character]

In [ ]:
schema = Company.model_json_schema()

### Generator korzystający ze schema z Pydantic

In [ ]:
# generator with structured output
generator = generate.json(model, Company)

Compiling FSM index for all state transitions: 100%|██████████| 119/119 [00:00<00:00, 197.12it/s]


### Dla porównania spróbujemy zmusić model do odpowiadania w formacie JSON także samym promptem

In [ ]:
# generator controiled with prompting only
generator_unstructured = generate.text(model)

### Prompt + JSON schema

In [ ]:
chat = [
    {"role": "system", "content": f"""
Jesteś modelem AI odpowiającym na pytania używając tylko formatu JSON

Zastosuj się do podanego JSON schema:\n{Company.schema_json()}"""},
    {"role": "user", "content": f"{text}"},
]

prompt = f"{tokenizer.apply_chat_template(chat, tokenize=False)}<|im_start|>assistant\n<schema>"

In [ ]:
markdown_content = f'### Cały prompt po użyciu tokenizera i templatki chatu wygląda tak:\n---\n ```text\n{prompt}\n```'

# Display the Markdown content
display(Markdown(markdown_content))

### Cały prompt po użyciu tokenizera i templatki chatu wygląda tak:
---
 ```text
<s><|im_start|>system

Jesteś modelem AI odpowiającym na pytania używając tylko formatu JSON

Zastosuj się do podanego JSON schema:
{"$defs": {"Character": {"properties": {"name": {"$ref": "#/$defs/CharactersNames"}, "items": {"items": {"$ref": "#/$defs/Item"}, "title": "Items", "type": "array"}}, "required": ["name", "items"], "title": "Character", "type": "object"}, "CharactersNames": {"enum": ["Frodo", "Sam", "Merry"], "title": "CharactersNames", "type": "string"}, "Item": {"properties": {"name": {"title": "Name", "type": "string"}, "quantity": {"anyOf": [{"type": "integer"}, {"type": "string"}, {"type": "null"}], "title": "Quantity"}}, "required": ["name", "quantity"], "title": "Item", "type": "object"}}, "properties": {"characters": {"items": {"$ref": "#/$defs/Character"}, "title": "Characters", "type": "array"}}, "required": ["characters"], "title": "Company", "type": "object"}<|im_end|>
<|im_start|>user

Każdy z hobbitów miał plecak, a w nim najpotrzebniejsze rzeczy. 
Frodo niósł trzy zapasowe koszule, dwa płaszcze i parę koców. 
Sam miał cztery bochenki chleba, sześć kawałków suszonego mięsa i kilka ciastek, 
a Merry wziął bukłak z wodą, dwie latarnie i dziesięć świec. 
<|im_end|>
<|im_start|>assistant
<schema>
```

### Do próby zmuszenia modelu do odpowiedzenia w takim formacie jak chcemy bez Outlines użyjemy inny prompt, z bardziej szczegółową instrukcją

In [ ]:
chat_unstructured = [
  {"role": "system", "content": """
# ZADANIE:
Z danego tekstu wybierz postacie i ich ekwipunek.
## Przykład:
### Tekst
Bilbo miał dwa jabłka a Frodo miecz.
### Odpowiedź
```
{
    "characters": [
        {
            "name": "Bilbo",
            "items": [
                {
                    "name": "jabłko",
                    "quantity": 2
                }
            ]
        },
        {
            "name": "Frodo",
            "items": [
                {
                    "name": "miecz",
                    "quantity": 1
                }
            ]
        },
    ]
}
```

Odpowiadaj tylko w formacie JSON, nie dodawaj nic więcej."""},
  {"role": "user", "content": f"# TEKST: \n{ text }"},
]

prompt_unstructured = f"{tokenizer.apply_chat_template(chat_unstructured, tokenize=False)}<|im_start|>assistant\n# JSON:"

### Dla porównania wygenerujemy rezultat bez narzucania struktury outputu

In [ ]:
response_unstructured = generator_unstructured(prompt_unstructured, max_tokens=1024)

Processed prompts: 100%|██████████| 1/1 [00:11<00:00, 11.85s/it, est. speed input: 31.73 toks/s, output: 32.48 toks/s]


In [ ]:
markdown_content = f'### Response bez wymuszenia struktury:\n---\n{response_unstructured}\n'

display(Markdown(markdown_content))

### Response bez wymuszenia struktury:
---


```json
{
  "characters": [
    {
      "name": "Każdy z hobbitów",
      "items": [
        {
          "name": "plecak",
          "quantity": 1
        }
      ]
    },
    {
      "name": "Frodo",
      "items": [
        {
          "name": "zapasowe koszule",
          "quantity": 3
        },
        {
          "name": "płaszcze",
          "quantity": 2
        },
        {
          "name": "koce",
          "quantity": 1
        }
      ]
    },
    {
      "name": "Sam",
      "items": [
        {
          "name": "bochenki chleba",
          "quantity": 4
        },
        {
          "name": "suszonego mięsa",
          "quantity": 6
        },
        {
          "name": "ciastka",
          "quantity": 1
        }
      ]
    },
    {
      "name": "Merry",
      "items": [
        {
          "name": "bukłak z wodą",
          "quantity": 1
        },
        {
          "name": "latarnie",
          "quantity": 2
        },
        {
          "name": "świece",
          "quantity": 10
        }
      ]
    }
  ]
}
```


O ile da się zmusić model do odpowiedzi, nie jest to deterministyczny mechanizm i output nie jest gwarantowany.  

Trudno go wykorzystać w produkcyjnych warunkach.

Model dodaje tez czasem tekst poza odpowiedzią w formacie JSON

### I kontrolując strukturę przez Outlines i Pydantic

In [ ]:
response = generator(prompt, max_tokens=1024)

Processed prompts: 100%|██████████| 1/1 [00:06<00:00,  6.15s/it, est. speed input: 71.66 toks/s, output: 31.36 toks/s]


In [ ]:
json_str = response.model_dump_json(indent=4)

markdown_content = f'### Response gdy output ma wymuszoną strukturę, obiekt Pydantic po konwersji do JSON:\n---\n```json\n{json_str}\n```'

display(Markdown(markdown_content))

### Response gdy output ma wymuszoną strukturę, obiekt Pydantic po konwersji do JSON:
---
```json
{
    "characters": [
        {
            "name": "Frodo",
            "items": [
                {
                    "name": "koszula",
                    "quantity": 3
                },
                {
                    "name": "płaszcz",
                    "quantity": 2
                },
                {
                    "name": "koce",
                    "quantity": 1
                }
            ]
        },
        {
            "name": "Sam",
            "items": [
                {
                    "name": "bochenki chleba",
                    "quantity": 4
                },
                {
                    "name": "suszone mięso",
                    "quantity": 6
                },
                {
                    "name": "ciastka",
                    "quantity": "kilka"
                }
            ]
        },
        {
            "name": "Merry",
            "items": [
                {
                    "name": "bukłak z wodą",
                    "quantity": 1
                },
                {
                    "name": "latarnie",
                    "quantity": 2
                },
                {
                    "name": "świece",
                    "quantity": 10
                }
            ]
        }
    ]
}
```

### Z Outlines rezultat ma gwarantowaną strukturę